In [ ]:
from itertools import islice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from tqdm import tqdm 
import sys
from datetime import datetime, timedelta, time
from scipy.optimize import curve_fit
from scipy import stats

# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

# 親ディレクトリをsys.pathに追加
sys.path.append(os.path.dirname(os.getcwd()))
from utils.point_history_utils import replace_nan, set_dtype, parse_date

# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'


# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)


In [ ]:
def aggregate_shop_date(df):
    # shop_idと年月日ごとにグループ化し、合計値と代表値を計算
    aggregated_df = df.groupby(['shop_id', '年月日']).agg({
        'amount': 'sum',
        'amount_kg': 'sum',
        'point': 'sum',
        'total_point': 'sum',
        'total_amount': 'sum',
        'coin': 'sum',
        'series_id': 'first',
        'shop_name': 'first',
        'リサイクル分類ID': 'first',
        '支店ID': 'first',
        'super': 'first',
        'prefectures': 'first',
        'municipality': 'first',
        'shop_name_1': 'first',
        'shop_id_1': 'first',
        'store_opening_time': 'first',
        'store_closing_time': 'first',
        'rps_opening_time': 'first',
        'rps_closing_time': 'first',
        'store_latitude': 'first',
        'store_longitude': 'first',
        '天気': 'first',
        '平均気温(℃)': 'first',
        '最高気温(℃)': 'first',
        '最低気温(℃)': 'first',
        '降水量の合計(mm)': 'first',
        '平均風速(m/s)': 'first',
        '平均湿度(％)': 'first',
        '平均現地気圧(hPa)': 'first',
        '平均雲量(10分比)': 'first',
        '降雪量合計(cm)': 'first',
        '日照時間(時間)': 'first',
        '合計全天日射量(MJ/㎡)': 'first'
    }).reset_index()

    # shop_idと年月日でソート
    aggregated_df = aggregated_df.sort_values(by=['shop_id', '年月日'])

    # 結果を保存
    aggregated_df.to_csv('data/input/point_history_per_shop_date.csv', index=False, encoding="utf-8")

def aggregate_date(df):
    # shop_idごとにグループ化し、合計値と代表値を計算
    aggregated_df = df.groupby(['shop_id']).agg({
        'amount': 'sum',
        'amount_kg': 'sum',
        'point': 'sum',
        'total_point': 'sum',
        'total_amount': 'sum',
        'coin': 'sum',
        'series_id': 'first',
        'shop_name': 'first',
        'リサイクル分類ID': 'first',
        '支店ID': 'first',
        'super': 'first',
        'prefectures': 'first',
        'municipality': 'first',
        'shop_name_1': 'first',
        'shop_id_1': 'first',
        'store_opening_time': 'first',
        'store_closing_time': 'first',
        'rps_opening_time': 'first',
        'rps_closing_time': 'first',
        'store_latitude': 'first',
        'store_longitude': 'first',
        '天気': 'first',
        '平均気温(℃)': 'first',
        '最高気温(℃)': 'first',
        '最低気温(℃)': 'first',
        '降水量の合計(mm)': 'first',
        '平均風速(m/s)': 'first',
        '平均湿度(％)': 'first',
        '平均現地気圧(hPa)': 'first',
        '平均雲量(10分比)': 'first',
        '降雪量合計(cm)': 'first',
        '日照時間(時間)': 'first',
        '合計全天日射量(MJ/㎡)': 'first'
    }).reset_index()

    # shop_idでソート
    aggregated_df = aggregated_df.sort_values(by=['shop_id'])

    # 結果を保存
    aggregated_df.to_csv('data/input/point_history_per_shop.csv', index=False, encoding="utf-8")


In [ ]:

#concat_csv()
#df = pd.read_csv('data/input/shop_data/point_history_ヨークベニマル_明石台店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/shop_data/point_history_ヨークベニマル_南中山店.csv', encoding="utf-8")
df = pd.read_csv('data/input/shop_data/point_history_みやぎ生協_加賀野店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/point_history_みやぎ生協_石巻大橋店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/point_history_みやぎ生協_加賀野店2.csv', encoding="utf-8")
#df = pd.read_csv('data/input/point_history_ビフレ_東通店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/shop_data/point_history_ヨークベニマル_小野町店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/shop_data/point_history_サン・マルシェ_大河原店.csv', encoding="utf-8")
#df = pd.read_csv('data/input/shop_data/point_history_清水フードセンター_西内野店(登録不可).csv', encoding="utf-8")
df = set_dtype(df)
df = replace_nan(df)
#df['rps_opening_time'] = pd.to_datetime(df['use_date'].dt.date.astype(str) + ' ' + df['rps_opening_time'])
#df['rps_closing_time'] = pd.to_datetime(df['use_date'].dt.date.astype(str) + ' ' + df['rps_closing_time'])
#df['rps_opening_time'] = pd.to_datetime(df['rps_opening_time'])
#df['rps_closing_time'] = pd.to_datetime(df['rps_closing_time'])
# df[(pd.to_datetime(df['use_date']) < pd.to_datetime('2023-01-02')) & (pd.to_datetime(df['use_date']) > pd.to_datetime('2022-12-30'))].sort_values(by='use_date')


In [ ]:
df[:1]

In [ ]:
# 年月日ごとにグループ化し、amount_kgの合計値をplot
df['年月日'] = pd.to_datetime(df['use_date']).dt.floor('d')
df.groupby('年月日')['amount_kg'].sum().plot()

In [ ]:
# use_date列の差分を計算
df['time_diff'] = df['use_date'].diff()

# df['年月日']について前の行と日付が異なる場合、df['rps_closing_time']とdf['rps_opening_time']の差をdf['time_diff']に格納
df.loc[df['年月日'].diff().dt.total_seconds() != 0, 'time_diff'] -=  df['rps_opening_time'] - df['rps_closing_time'].shift(1)


df['time_diff'] = df['time_diff'].dt.total_seconds() / 3600

# 最初の行には nan を設定
df.loc[0, 'time_diff'] = np.nan

In [ ]:
# df['use_date'].diff().dt.days != 0 がtrueのインデックスを取得
df[df['use_date'].dt.date.diff().dt.days != 0].index

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['use_date'],df['time_diff'])
#ax.scatter(df['use_date'],df['time_diff'], s=2)

# x軸のラベル表示間隔を調整
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

ax.set_xlabel('年月日')
ax.set_ylabel('リサイクルステーションの利用間隔[h]')

# yの最大値
ax.set_ylim(0, 6)



In [ ]:
# df['use_date_diff']の分布を確認
fig, ax = plt.subplots()
ax.hist(df['time_diff'], bins=100, range=(0, 12))
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('頻度')
# 両対数にする
ax.set_yscale('log')
ax.set_xscale('log')

In [ ]:
# べき乗則関数を定義
def power_law(x, a, b):
    return a * np.power(x, b)

# 指数関数を定義
def exp_func(x, a, b):
    return a*np.exp(-b*x)
    #return a**(-b*x)

# ヒストグラムのデータを取得
counts, bin_edges = np.histogram(df['time_diff'], bins=100, range=(0, 12))
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# x軸の大きい値を重視してべき乗則のフィットを行う
mask = counts > 0
weights = np.ones(shape = bin_centers[mask].shape)
weights = 1 / bin_centers[mask] ** 2
#weights = 10e4 * np.e ** (-bin_centers[mask])

# 重み付きフィットを実行
#params, params_covariance = curve_fit(power_law, bin_centers[mask], counts[mask], sigma=weights)
params, params_covariance = curve_fit(exp_func, bin_centers[mask], counts[mask], sigma=weights)
print("params",params)

# フィット結果をプロット
fig, ax = plt.subplots()
ax.bar(bin_centers, counts, width=np.diff(bin_edges), label='Data')
#ax.plot(bin_centers, power_law(bin_centers, *params)+0.01, label='Fit: a=%.2f, b=%.2f' % tuple(params), color='red')
ax.plot(bin_centers, exp_func(bin_centers, *params)+0.01, label='Fit: a=%.2f, b=%.2f' % tuple(params), color='red')
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('Frequency')
ax.set_yscale('log')
ax.set_xscale('log')

In [ ]:
bin_centers[:10]

In [ ]:
# KS検定を実行して指数分布であるかを検定
lambda_est = 1.0 / np.mean(bin_edges)  # データからラムダを推定
d, p_value = stats.kstest( bin_edges, 'expon', args=(0, 1/lambda_est))

print(f"KS Statistic: {d}")
print(f"P-value: {p_value}")

In [ ]:
a, b = params[:2]
expected = exp_func(bin_centers,a,b)  # 各ビンでの期待頻度
#expected = a * np.power(bin_centers, b) * (bin_centers[1] - bin_centers[0])  # 各ビンでの期待頻度

index = 2
expected *= sum(counts[index:]) / sum(expected[index:])  # 期待頻度を正規化
# カイ二乗適合度検定
chi_squared_stat, p_value = stats.chisquare(counts[index:], f_exp=expected[index:])

print(f"Chi-squared statistic: {chi_squared_stat}")
print(f"P-value: {p_value}")

# ヒストグラムと期待される分布のプロット
fig, ax = plt.subplots() 
ax.bar(bin_centers, counts, width=np.diff(bin_edges), label='Data')
ax.plot(bin_centers, exp_func(bin_centers,a,b)+0.01, linewidth=2, color='r')
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('Frequency')
ax.set_yscale('log')
ax.set_xscale('log')
plt.show()

In [ ]:
# 閾値を設定（x軸が小さい部分は誤差が大きいため除外）
threshold = np.percentile(bin_centers[mask], 10)

# 閾値より大きい値のみを考慮
selected_mask = bin_centers[mask] >= threshold

# 残差平方和（RSS）を計算
residuals = counts[mask][selected_mask] - power_law(bin_centers[mask][selected_mask], *params)
rss = np.sum(residuals**2)

# 全変動平方和（TSS）を計算
mean_counts = np.mean(counts[mask][selected_mask])
tss = np.sum((counts[mask][selected_mask] - mean_counts)**2)

# 決定係数（R²）を計算
r_squared = 1 - (rss / tss)

print(f"R-squared: {r_squared}")